In [358]:
import pandas as pd
import numpy as np

from scipy.optimize import curve_fit

import plotly.express as px

In [359]:
dtypes = {
    'z': 'UInt8',
    'n': 'UInt8',
    'symbol': 'string',
    'idx': 'UInt16',
    'energy_shift': 'category',
    'energy': 'Float64',
    'unc_e': 'Float64',
    'ripl_shift': 'Float64',
    'jp': 'string',
    'jp_order': 'UInt8',
    'half_life': 'string',
    'operator_hl': 'string',
    'unc_hl': 'string',
    'unit_hl': 'category',
    'half_life_sec': 'Float64',
    'unc_hl.1': 'Float64',
    'decay_1': 'string',
    'decay_1_%': 'Float64',
    'unc_1': 'Float64',
    'decay_2': 'string',
    'decay_2_%': 'Float64',
    'unc_2': 'Float64',
    'decay_3': 'string',
    'decay_3_%': 'Float64',
    'unc_3': 'Float64',
    'isospin': 'string',
    'magnetic_dipole': 'Float64',
    'unc_mn': 'Float64',
    'electric_quadrupole': 'Float64',
    'unc_eq': 'Float64',
    'ENSDF_publication_cut-off': 'string',
    'ENSlevels_df_authors': 'string',
    'Extraction_date': 'string'
}
parse_dates = ['ENSDF_publication_cut-off', 'Extraction_date']

In [360]:
# the service URL
livechart = "https://nds.iaea.org/relnsd/v1/data?"

# There have been cases in which the service returns an HTTP Error 403: Forbidden
# use this workaround
import urllib.request
def lc_pd_dataframe(url, dtype=None, parse_dates=None):
    req = urllib.request.Request(url)
    req.add_header('User-Agent''','' 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req), dtype=dtype, parse_dates=parse_dates)

ground_states_df = lc_pd_dataframe(livechart + "fields=ground_states&nuclides=all", dtype=dtypes)
ground_states_df = ground_states_df[['z', 'n', 'symbol', 'half_life', 'electric_quadrupole']]
ground_states_df = ground_states_df[ground_states_df['half_life'] == 'STABLE']
ground_states_df = ground_states_df.copy()
ground_states_df['a'] = ground_states_df['z'] + ground_states_df['n']
ground_states_df = ground_states_df.set_index(['z', 'a'], drop=True)

## Load saved data

In [ ]:
levels_df = pd.read_csv('levels.csv', parse_dates=parse_dates, dtype=dtypes)


levels_df.loc[levels_df['half_life'] == 'STABLE', 'half_life'] = 'inf'
levels_df['half_life'] = levels_df['half_life'].astype('Float64')

levels_df['a'] = levels_df['z'] + levels_df['n']

levels_df = levels_df[['symbol', 'a', 'z', 'n', 'idx', 'energy', 'energy_shift', 'ripl_shift', 'jp', 'jp_order']]
levels_df['energy'] = levels_df['energy'] / 1000 #MeV
levels_df['ripl_shift'] = levels_df['ripl_shift'] / 1000 #MeV
#TODO also take into account energy uncertainty

levels_df['beta'] = (levels_df['n'].astype('Float64') - levels_df['z'].astype('Float64')) / levels_df['a'].astype('Float64')

levels_df = levels_df.set_index(['z', 'a'], drop=True)

symbol  n  idx  energy energy_shift  ripl_shift    jp  jp_order      beta
z a                                                                          
1 1      H  0    0     0.0          NaN        <NA>  1/2+         1      -1.0
  2      H  1    0     0.0          NaN        <NA>    1+         1       0.0
2 3     He  1    0     0.0          NaN        <NA>  1/2+         1 -0.333333
  4     He  2    0     0.0          NaN        <NA>    0+         1       0.0
  4     He  2    1   20.21          NaN        <NA>    0+         2       0.0

In [ ]:
# Filter out unknown energies

levels_df = levels_df[levels_df['energy_shift'].isna()]
levels_df = levels_df.drop(['energy_shift', 'ripl_shift'], axis='columns')

symbol  n  idx  energy    jp  jp_order      beta
z a                                                 
1 1      H  0    0     0.0  1/2+         1      -1.0
  2      H  1    0     0.0    1+         1       0.0
2 3     He  1    0     0.0  1/2+         1 -0.333333
  4     He  2    0     0.0    0+         1       0.0
  4     He  2    1   20.21    0+         2       0.0

In [ ]:
# Keep only even-even nuclei

df = levels_df.join(ground_states_df[['electric_quadrupole']])
df = df[(df.index.get_level_values('z') % 2 == 0) & (df.index.get_level_values('a') % 2 == 0)]
df

symbol    n  idx  energy    jp  jp_order      beta  electric_quadrupole
z  a                                                                          
2  4       He    2    0     0.0    0+         1       0.0                 <NA>
   4       He    2    1   20.21    0+         2       0.0                 <NA>
   4       He    2    2   21.01    0-         1       0.0                 <NA>
   4       He    2    3   21.84    2-         1       0.0                 <NA>
   4       He    2    4   23.33    2-         2       0.0                 <NA>
...       ...  ...  ...     ...   ...       ...       ...                  ...
82 208     Pb  126  606    22.1    1-        40  0.211538                 <NA>
   208     Pb  126  607   23.94  <NA>      <NA>  0.211538                 <NA>
   208     Pb  126  608   24.48  <NA>      <NA>  0.211538                 <NA>
   208     Pb  126  609    26.9    2+        62  0.211538                 <NA>
   208     Pb  126  610    27.2  <NA>      <NA>  0.211538                 <NA>

[29233 rows x 8 columns]

In [365]:
quadrupoles = df.groupby(level=['z','a'])['electric_quadrupole'].mean()
print(f"Missing {quadrupoles.isna().sum()} / {len(quadrupoles)} = {quadrupoles.isna().sum() / len(quadrupoles):.2f}")
quadrupoles = (quadrupoles - quadrupoles.mean()) / quadrupoles.std() #standardize

min_quad = quadrupoles.min()
max_quad = quadrupoles.max()
bound = np.max(np.abs([cmin, cmax]))

quadrupoles[quadrupoles.isna()] = -np.inf

quadrupoles = pd.DataFrame(quadrupoles).reset_index()
quadrupoles['n'] = quadrupoles['a'] - quadrupoles['z']


fig = px.scatter(quadrupoles,
          x='n',
          y='z', 
          color='electric_quadrupole',
          title='Missing quadrupole moments',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'quadrupole_moment': 'Quadrupole moment'},
          color_continuous_scale =[(0, 'black'),
                    (0.01, 'black'),
                    (0.01, 'yellow'),
                    (1, 'green')],
            range_color=(-bound, bound))


magic_numbers = [2, 8, 20, 50, 58, 82, 126]

for i, m in enumerate(magic_numbers):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()

quadrupoles.loc[quadrupoles['electric_quadrupole'] == -np.inf, 'electric_quadrupole'] = pd.NA


Missing 60 / 140 = 0.43


In [366]:
quadrupoles

,z,a,electric_quadrupole,n
0,2,4,<NA>,2
1,6,12,0.724178,6
2,8,16,<NA>,8
3,8,18,<NA>,10
4,10,20,0.390906,10
...,...,...,...,...
135,80,200,<NA>,120
136,80,202,1.655042,122
137,80,204,1.114911,124
138,82,206,0.712686,124


In [367]:
# Note 82 52 is very non-spherical, even though it is filled shell

In [368]:
fig = px.scatter(quadrupoles[quadrupoles['electric_quadrupole'].isna()],
          x='n',
          y='z', 
          title='Missing quadrupole moments',
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'quadrupole_moment': 'Quadrupole moment'})


magic_numbers = [2, 8, 20, 50, 58, 82, 126]

for i, m in enumerate(magic_numbers):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)
    fig.add_hline(y=m, line_dash="dash", line_color="gray")

fig.show()


In [369]:
quadrupoles = pd.merge(df, quadrupoles.drop(columns='n'), 
                        left_index=True, right_on=['z', 'a'], 
                        suffixes=('', '_standardized')
                    ).set_index(['z','a'])

In [370]:
# keep spherical only
spherical_df = df[quadrupoles['electric_quadrupole_standardized'].abs() <= 1]

In [371]:
# Filter out not enough levels
# Just removes 1H, 2H, and 3He

spherical_df = spherical_df.loc[spherical_df.groupby(level=spherical_df.index.names).size() >= 3]

# Filter out uncertain jp

spherical_df = spherical_df[spherical_df['jp'].str.fullmatch(r'^[0-9]+(/[0-9]+)?[+-]$')]

# Extract j and p

spherical_df['j'] = spherical_df['jp'].str[:-1]
spherical_df['p'] = spherical_df['jp'].str[-1]

odd_spins = spherical_df['j'].str.contains('/')
even_spins = ~odd_spins

spherical_df.loc[odd_spins,'j_float'] = spherical_df[odd_spins]['j'].str.split('/').apply(lambda x: float(x[0]) / float(x[1]))
spherical_df.loc[even_spins,'j_float'] = spherical_df[even_spins]['j'].astype(float)

spherical_df['j_evenness'] = pd.Series(data=pd.NA, dtype='boolean')
spherical_df.loc[even_spins,'j_evenness'] = (spherical_df[even_spins]['j'].str.split('/', expand=True)[0].astype(int) % 2 == 0)
#TODO similar for odd_spins

spherical_df['p_bit'] = spherical_df['p'].str.fullmatch(r'\-').astype(int)

# Search

## For quadrupole oscillator

### Quadrupole, positive, first band 

In [372]:
def get_osc_energy(n, energy_quantum, zero_point_energy):
    return n * energy_quantum + zero_point_energy

def get_rot_energy(l, energy_quantum, zero_point_energy):
    return l*(l+1) * energy_quantum + zero_point_energy #TODO remove zero point energy

def fit(group, func):

    x = group['quanta']
    y = group['energy']

    # fit
    popt, pcov = curve_fit(func, x, y)

    # prediction
    y_pred = func(x, *popt)

    # r-squared
    ss_res = np.sum((y - y_pred)**2)
    ss_tot = np.sum((y - y.mean())**2)
    r2 = 1 - (ss_res / ss_tot)

    # results
    results = pd.DataFrame({'energy_quantum': popt[0], 'zero_point_energy': popt[1], 'r2': r2,  'j_float': x, 'quanta': x, 'energy_pred': y_pred})

    # results = pd.Series(list(popt)+[r2], index=['energy_quantum', 'zero_point_energy', 'r2'])
    return  results

In [373]:
quad_pos_first = spherical_df[(spherical_df['jp_order'] == 1) & spherical_df['j_evenness'].fillna(False) & (spherical_df['p_bit'] == 0)]
quad_pos_first = quad_pos_first.loc[quad_pos_first.groupby(level=spherical_df.index.names).size() >= 3]
quad_pos_first['quanta'] = (quad_pos_first['j_float'].astype(int) // 2)

quad_pos_first_groups = quad_pos_first.groupby(level=spherical_df.index.names, as_index=False)

oscillator_fit = quad_pos_first_groups.apply(lambda group: fit(group, get_osc_energy)).droplevel(0)

In [374]:
min_r2 = 0.9

print("Oscillator osc:")
best_oscillator_fit = oscillator_fit[oscillator_fit['r2'] > min_r2]

print(best_oscillator_fit.groupby(by=best_oscillator_fit.index)['r2'].head(1))

Oscillator osc:
z   a  
6   12     0.964487
10  22     0.981052
12  24     0.938550
    26     0.949867
14  30     0.982564
16  34     0.990168
18  36     0.992320
    40     0.967876
20  42     0.939244
22  46     0.987631
    48     0.986369
24  52     0.958788
    54     0.985646
26  56     0.981031
    58     0.982224
28  58     0.957798
    60     0.993792
30  64     0.986505
    66     0.992357
    68     0.996080
32  70     0.997881
    74     0.988616
    76     0.989356
34  74     0.978128
    78     0.990448
42  96     0.991078
    98     0.999310
44  96     0.982977
46  102    0.978908
    104    0.997169
    106    0.995995
    108    0.988163
48  106    0.988761
    110    0.974336
    112    0.995564
    114    0.996796
50  112    0.995924
    114    0.991056
    116    0.971504
    122    0.963508
    124    0.955386
52  120    0.991651
    122    0.989891
    124    0.990797
54  130    0.992777
56  136    0.966790
60  146    0.997897
62  150    0.994442
68  164    0.965

In [375]:
#TODO plot r^2 on grid
#TODO change starting point

In [376]:
merged = quad_pos_first.merge(oscillator_fit, how='left', on=['z','a', 'quanta'])
best_osc_merged = merged[merged['r2'] > min_r2]

def get_best_r2(df, min_r2):
    return df[df['r2'] > min_r2]

# merged = merged.merge(rotator_fit, how='left', on=['z','a', 'idx'], suffixes=('_osc', '_rot'))
# lowest_merged = merged.groupby(['z','a']).head(3)
lowest_merged = merged
# best_rot_merged = lowest_merged[lowest_merged['r2_rot'] > min_r2]
# best_rot_merged

In [377]:
best_osc_merged

symbol    n  idx    energy   jp  jp_order      beta  \
z  a                                                         
6  12       C    6    0       0.0   0+         1       0.0   
   12       C    6    1   4.43982   2+         1       0.0   
   12       C    6   11      13.3   4+         1       0.0   
10 22      Ne   12    0       0.0   0+         1  0.090909   
   22      Ne   12    1  1.274537   2+         1  0.090909   
...       ...  ...  ...       ...  ...       ...       ...   
76 192     Os  116    3   0.58028   4+         1  0.208333   
   192     Os  116    8   1.08923   6+         1  0.208333   
   192     Os  116   23   1.70839   8+         1  0.208333   
   192     Os  116   68    2.4188  10+         1  0.208333   
   192     Os  116   98    3.2108  12+         1  0.208333   

        electric_quadrupole   j  p j_float_x  j_evenness  p_bit  quanta  \
z  a                                                                      
6  12                  0.06   0  +       0.0        True      0       0   
   12                  0.06   2  +       2.0        True      0       1   
   12                  0.06   4  +       4.0        True      0       2   
10 22                -0.215   0  +       0.0        True      0       0   
   22                -0.215   2  +       2.0        True      0       1   
...                     ...  .. ..       ...         ...    ...     ...   
76 192                -0.96   4  +       4.0        True      0       2   
   192                -0.96   6  +       6.0        True      0       3   
   192                -0.96   8  +       8.0        True      0       4   
   192                -0.96  10  +      10.0        True      0       5   
   192                -0.96  12  +      12.0        True      0       6   

        energy_quantum  zero_point_energy        r2  j_float_y  energy_pred  
z  a                                                                         
6  12         6.650000          -0.736727  0.964487          0    -0.736727  
   12         6.650000          -0.736727  0.964487          1     5.913273  
   12         6.650000          -0.736727  0.964487          2    12.563273  
10 22         1.678600          -0.134688  0.981052          0    -0.134688  
   22         1.678600          -0.134688  0.981052          1     1.543912  
...                ...                ...       ...        ...          ...  
76 192        0.542376          -0.310942  0.967042          2     0.773809  
   192        0.542376          -0.310942  0.967042          3     1.316185  
   192        0.542376          -0.310942  0.967042          4     1.858561  
   192        0.542376          -0.310942  0.967042          5     2.400936  
   192        0.542376          -0.310942  0.967042          6     2.943312  

[356 rows x 19 columns]

In [378]:
merged.loc[24,52]

symbol   n  idx    energy   jp  jp_order      beta  electric_quadrupole  \
z  a                                                                            
24 52     Cr  28    0       0.0   0+         1  0.076923                -0.08   
   52     Cr  28    1  1.434091   2+         1  0.076923                -0.08   
   52     Cr  28    2   2.36963   4+         1  0.076923                -0.08   
   52     Cr  28    6  3.113858   6+         1  0.076923                -0.08   
   52     Cr  28   26   4.75031   8+         1  0.076923                -0.08   
   52     Cr  28  125    7.2379  10+         1  0.076923                -0.08   

        j  p j_float_x  j_evenness  p_bit  quanta  energy_quantum  \
z  a                                                                
24 52   0  +       0.0        True      0       0        1.339497   
   52   2  +       2.0        True      0       1        1.339497   
   52   4  +       4.0        True      0       2        1.339497   
   52   6  +       6.0        True      0       3        1.339497   
   52   8  +       8.0        True      0       4        1.339497   
   52  10  +      10.0        True      0       5        1.339497   

       zero_point_energy        r2  j_float_y  energy_pred  
z  a                                                        
24 52          -0.197777  0.958788          0    -0.197777  
   52          -0.197777  0.958788          1     1.141720  
   52          -0.197777  0.958788          2     2.481216  
   52          -0.197777  0.958788          3     3.820713  
   52          -0.197777  0.958788          4     5.160210  
   52          -0.197777  0.958788          5     6.499707

In [379]:
df_to_plot = best_osc_merged
# df_to_plot = merged

fig = px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
          # x='a', 
          x='n',
          y='z', 
          color='r2',
          title='R² Values for Best Oscillator Fits',
          # labels={'z': 'Number of protons (Z)', 'a': 'Weight (A)', 'r2': 'R² Score'},
          labels={'z': 'Number of protons (Z)', 'n': 'Number of neutrons (N)', 'r2': 'R² Score'},
          color_continuous_scale='RdBu',
          height=800)


magic_numbers = [2, 8, 20, 50, 58, 82, 126]

for i, m in enumerate(magic_numbers):
    fig.add_vline(x=m, line_dash="dash", line_color="gray", name="Magic numbers" if i==0 else None)

    fig.add_hline(y=m, line_dash="dash", line_color="gray")
    # fig.add_scatter(x=[m,200], y=[0,200-m], mode='lines', line_dash="dash", line_color='gray', showlegend=i==0, name="Magic numbers" if i==0 else None)

fig.show()

In [396]:
df_to_plot = merged

px.scatter(df_to_plot.groupby(by=df_to_plot.index).head(1).reset_index(), 
            x='electric_quadrupole', 
            y='r2',
            title='R² Values wrt sphericality',
            labels={'electric_quadrupole': 'Electric quadrupole (Q)', 'a': 'Weight (A)', 'r2': 'R² Score'},
            color_continuous_scale='viridis')

In [381]:
merged.loc[24,52]

symbol   n  idx    energy   jp  jp_order      beta  electric_quadrupole  \
z  a                                                                            
24 52     Cr  28    0       0.0   0+         1  0.076923                -0.08   
   52     Cr  28    1  1.434091   2+         1  0.076923                -0.08   
   52     Cr  28    2   2.36963   4+         1  0.076923                -0.08   
   52     Cr  28    6  3.113858   6+         1  0.076923                -0.08   
   52     Cr  28   26   4.75031   8+         1  0.076923                -0.08   
   52     Cr  28  125    7.2379  10+         1  0.076923                -0.08   

        j  p j_float_x  j_evenness  p_bit  quanta  energy_quantum  \
z  a                                                                
24 52   0  +       0.0        True      0       0        1.339497   
   52   2  +       2.0        True      0       1        1.339497   
   52   4  +       4.0        True      0       2        1.339497   
   52   6  +       6.0        True      0       3        1.339497   
   52   8  +       8.0        True      0       4        1.339497   
   52  10  +      10.0        True      0       5        1.339497   

       zero_point_energy        r2  j_float_y  energy_pred  
z  a                                                        
24 52          -0.197777  0.958788          0    -0.197777  
   52          -0.197777  0.958788          1     1.141720  
   52          -0.197777  0.958788          2     2.481216  
   52          -0.197777  0.958788          3     3.820713  
   52          -0.197777  0.958788          4     5.160210  
   52          -0.197777  0.958788          5     6.499707

In [382]:
test_nucleus = merged.loc[24,52]
a = test_nucleus.index[0][1]
symbol = test_nucleus.iloc[0]['symbol']

px.scatter(x=test_nucleus['quanta'], 
            y=test_nucleus['energy']-test_nucleus['energy_pred'],
            title=f'Residuals for {a}{symbol}',
            labels={'x': 'Quanta of oscillation', 'y': 'Residuals'})

In [383]:
residuals_df = best_osc_merged.copy()
residuals_df = residuals_df.reset_index()
residuals_df["name"] = residuals_df["a"].astype(str) + residuals_df["symbol"]
residuals_df["residuals"] = residuals_df["energy"] - residuals_df["energy_pred"]

best_residuals = get_best_r2(residuals_df, 0.99)

fig = px.scatter(best_residuals, x="quanta", y="residuals", facet_col="name", facet_col_wrap=8)
fig.add_hline(y=0, line_width=1, line_dash="dash", line_color="black")
fig.show()

In [384]:
#What if any fit, not linear?
# Note some seem to have two different bands, (W shape)
# Note test multiple models, and only show best fit


In [385]:
test_nucleus = residuals_df[residuals_df['name'] == '60Ni']

fig = px.scatter(test_nucleus, x="quanta", y="energy")
fig.add_scatter(x=test_nucleus["quanta"], 
                y=(test_nucleus["energy_quantum"] * test_nucleus["quanta"] + test_nucleus['zero_point_energy']), 
                mode='lines', line_width=1, line_dash="dash", line_color="black")
fig.show()

In [386]:
residuals_df[residuals_df['r2'] == residuals_df['r2'].min()][['z', 'a']]

,z,a
6,12,24
7,12,24
8,12,24
9,12,24
10,12,24


In [387]:
test_nucleus = residuals_df.loc[6:10]

fig = px.scatter(test_nucleus, x="quanta", y="energy")
fig.add_scatter(x=test_nucleus["quanta"], 
                y=(test_nucleus["energy_quantum"] * test_nucleus["quanta"] + test_nucleus['zero_point_energy']), 
                mode='lines', line_width=1, line_dash="dash", line_color="black")
fig.show()

In [388]:
group = test_nucleus
func = lambda n,w: n*w

x = group['quanta']
y = group['energy']

# fit
popt, pcov = curve_fit(func, x, y)

# prediction
y_pred = func(x, *popt)

# r-squared
ss_res = np.sum((y - y_pred)**2)
ss_tot = np.sum((y - y.mean())**2)
r2 = 1 - (ss_res / ss_tot)

# results
results = pd.DataFrame({'energy_quantum': popt[0], 'r2': r2,  'j_float': x, 'quanta': x, 'energy_pred': y_pred})


fig = px.scatter(group, x="quanta", y="energy")
fig.add_scatter(x=result["quanta"], y=result["energy_pred"],
                mode='lines', line_width=1, line_dash="dash", line_color="black")
fig.show()

In [389]:
result = fit(test_nucleus, get_rot_energy)

fig = px.scatter(test_nucleus, x="quanta", y="energy")
fig.add_scatter(x=result["quanta"], y=result["energy_pred"],
                mode='lines', line_width=1, line_dash="dash", line_color="black")
fig.show()

In [390]:
print(test_nucleus)
print(result)

     z   a symbol   n  idx    energy  jp  jp_order  beta  electric_quadrupole  \
6   12  24     Mg  12    0       0.0  0+         1   0.0                -0.29   
7   12  24     Mg  12    1  1.368667  2+         1   0.0                -0.29   
8   12  24     Mg  12    2  4.122853  4+         1   0.0                -0.29   
9   12  24     Mg  12   13    8.1132  6+         1   0.0                -0.29   
10  12  24     Mg  12  192     14.15  8+         1   0.0                -0.29   

    ... j_evenness p_bit quanta  energy_quantum  zero_point_energy       r2  \
6   ...       True     0      0        3.504453          -1.457963  0.93855   
7   ...       True     0      1        3.504453          -1.457963  0.93855   
8   ...       True     0      2        3.504453          -1.457963  0.93855   
9   ...       True     0      3        3.504453          -1.457963  0.93855   
10  ...       True     0      4        3.504453          -1.457963  0.93855   

    j_float_y  energy_pred  name  resi

In [391]:
#TODO compare with paper, check their r2 for "perfect" and for "deviation"
#TODO look at paper?

In [392]:
# Using r2 not mse because scale independent, some energy levels may be inherently larger?
#https://stats.stackexchange.com/a/250735

In [393]:
#TODO figure out way to combine r2 and number of points used

In [ ]:
#Quadrupole axial: Even positive only?

#TODO use bands


#TODO take into account energy shifts with ripl_shift
#TODO take into account energy shifts without ripl_shift